These data come from a survey conducted by INSEE, Budget des Familles, which are protected by the Statistical Secrecy Committee. To reproduce the analysis, one must request access to this data.

In [1]:
import pandas as pd

In [2]:
date_analysis = "2023-01"

In [3]:
df = pd.read_csv('BDF/Csv/C05.csv', sep=";", index_col="IDENT_MEN")

We need to remove the columns starting with 13 (taxes) and 14 (subsidies) which are not consumption. And also the columns total CTOT and pondmen (weight household) which are not useful.

In [4]:
columns_to_delete = df.columns[df.columns.str.startswith('C13') | df.columns.str.startswith('C14')]
df = df.drop(columns=columns_to_delete)
df = df.drop(columns=["CTOT", "pondmen"])

In [5]:
df_prop = df.divide(df.sum(axis=1), axis=0)


In [6]:
df_prop.rename(columns=lambda x: x.replace('C', ''), inplace=True)


We know have the proportion of each consumption area per household.

In [7]:
import eurostat

In [8]:
df_inflation_raw = eurostat.get_data_df(code="PRC_HICP_MANR", filter_pars={
                                    'geo': 'FR', 'startPeriod': "2021-03"})
df_inflation_raw = df_inflation_raw.rename(columns={'geo\TIME_PERIOD': 'geo'})

Dataset not found: PRC_HICP_MANR


ValueError: 

In [ ]:
df_correspondances = pd.read_csv("correspondances.csv", dtype="str", sep=";")

In [ ]:
df_correspondances.set_index('colonnes de df_prop', inplace=True)

In [ ]:
df_inflation = df_inflation_raw[df_inflation_raw['coicop'].str.startswith('CP')]

In [ ]:
df_inflation['coicop'] = df_inflation['coicop'].str.replace('CP', '')


C:\Users\mdaversa\AppData\Local\Temp\ipykernel_12052\2914206753.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inflation['coicop'] = df_inflation['coicop'].str.replace('CP', '')


In [ ]:
df_inflation = df_inflation.loc[:, ["coicop", date_analysis]]


In [ ]:
df_inflation = df_inflation.set_index('coicop')

In [ ]:
def cell_func(cell_value, col_name):
    # Votre logique ici, vous pouvez utiliser 'col_name' et 'cell_value'
    coicop = df_correspondances.loc[col_name]['colonnes de df_inflation correspondante']
    return df_inflation.loc[coicop][date_analysis]*cell_value

def col_func(col):
    column_name = col.name
    return col.apply(cell_func, args=(column_name,))


df_inflations = df_prop.apply(col_func)

In [ ]:
df_inflation_by_household = pd.DataFrame(df_inflations.sum(axis=1))

In [ ]:
df_inflation_by_household = df_inflation_by_household.rename(columns={0: "inflation"})

In [ ]:
df_inflation_by_household.to_csv(f"BDF/computed_inflation_by_household_{date_analysis}.csv")